# Lectures 7 & 8: Building Autograd engine from scratch 

## Learning outcomes
1. Understanding Automated Differentiation Engines at a foundational level
2. Operator Overloading in Object Oriented Programming
3. Graph Representation
4. Graph Traversal

--------

## Automated Differentiation from Scratch as a Concept
Automated differentiation is a key technique used in deep learning for efficiently computing gradients of complex functions. In this context, "gradients" refers to the partial derivatives of a function with respect to its inputs or parameters, which are used to update the parameters during the training process via gradient descent or similar optimization algorithms.

There are two main approaches to implementing automated differentiation: forward mode and backward mode. Forward mode computes the derivatives of a function with respect to all of its inputs simultaneously, while backward mode computes the derivatives of the function with respect to a single input at a time. In practice, backward mode is generally more efficient and widely used in deep learning frameworks such as PyTorch and TensorFlow.

To implement backward mode in Python from scratch, we can use a technique called "reverse-mode differentiation", also known as "backpropagation". Backpropagation works by breaking down a complex function into a series of simple, elementary operations, and then computing the derivative of the output with respect to each input in reverse order using the chain rule of calculus.


---------

## Problem Statement
Restrictin yourself to <font color='green'>Python's Standard Library</font>, build an <font color='green'>autograd engine</font> capable of estimating the gradients required to solve the following problem using gradient descent.

>> Find a point in the R<sup>2</sup> with the least average Euclidean distance to a set of arbitrary points. 

## Setting Up the dataset and defining the loss function

In [5]:
from random import Random
from math import sqrt
SEED = 5
def generate_pnts(N=1000):
     random_gen = Random(x=SEED)
     datax,datay=[],[]
     for _ in range (N):
          datax.append(random_gen.uniform(a=0,b=1))
     for _ in range (N):
          datay.append(random_gen.uniform(a=0,b=1))
     return datax,datay

def calc_grad(x_p,y_p,batch_x,batch_y):
          sum_x=0;sum_y=0
          # iterate over the elements of batch_x and batch_y in parallel
          for x_i,y_i in zip(batch_x,batch_y):
               # calculate the inverse square root of the distance between (x_i, y_i) and (x_p, y_p)
              invsqrt = ((x_i-x_p)**2+(y_i-y_p)**2)**(-0.5)
              # add the scaled difference between x_i and x_p to sum_x
              sum_x+=invsqrt*(x_i-x_p);
              # add the scaled difference between y_i and y_p to sum_y
              sum_y+=invsqrt*(y_i-y_p);
          return -sum_x/len(batch_x), -sum_y/len(batch_y)

def loss(x_p , y_p,dx,dy):
  # return the average of the square root of the sum of the squared differences
  # between x_p and each element of dx and y_p and each element of dy
     return (1/len(dx))*sum(
         [sqrt((x_i - x_p)**2 + (y_i - y_p)**2 )
         for x_i,y_i in zip(dx,dy)
          ]
     )

data_x,data_y = generate_pnts(N=1)
x_p,y_p = 0.3,0.3
# call the calc_grad function with x_p, y_p, data_x, and data_y as arguments and store the results in grad_x and grad_y
grad_x,grad_y = calc_grad(x_p,y_p,data_x,data_y)
# call the loss function with x_p, y_p, data_x, and data_y as arguments and store the result in cur_loss
cur_loss = loss(x_p,y_p,data_x,data_y)
print(f"Closed Form: gradient for x_p { grad_x}, gradient for y_p {grad_y}")
print(f"Closed Form: loss = {cur_loss}")



     

Closed Form: gradient for x_p -0.5900849147094943, gradient for y_p -0.8073411877467226
Closed Form: loss = 0.5472122517293468


## Pytorch as an example of Autograd engines

In [6]:
import torch 
import numpy as np
# Create a tensor with the initial point values and enable gradient tracking
pnt = torch.tensor([0.3,0.3])
pnt.requires_grad = True
# Retain gradients for the tensor for future computations
pnt.retain_grad()
data_x,data_y = generate_pnts(N=1000)
# Convert data to tensor and transpose it
data = torch.tensor([data_x,data_y])
# print(data.shape)  --> torch.Size([2, 1000])
# Transposing is done for the industry convention
# and to be able to broadcast pnt to [2,1000] and do subtraction
data = data.t()  
# print(data.shape,pnt.shape) --> torch.Size([1000, 2]) torch.Size([2])
for i in range(1):
      print("Iteration:",i)
      loss_torch =  torch.mean(torch.sqrt(((data-pnt)**2).sum(dim=1)))
      print(f"torch_Loss :{loss_torch}")
      # Compute gradients of the loss with respect to the tensor's values
      loss_torch.backward()
      print(f"torch_grad:{pnt.grad.data}")
      pnt.grad.zero_()

      


Iteration: 0
torch_Loss :0.44305282831192017
torch_grad:tensor([-0.3278, -0.3363])



This code uses PyTorch library to compute the loss and its gradients for a given point pnt.


* Import the PyTorch library
* Create a PyTorch tensor named pnt with the initial point values (0.3, 0.3),
* and enable gradient tracking for it
* Retain the gradients for the tensor pnt
* Generate 1000 random data points using the generate_pnts() function
* Convert data points into a PyTorch tensor and transpose it for future computations
* Calculate the loss using PyTorch's mean() and sqrt() functions
* Print the loss calculated using PyTorch's functions
* Compute gradients of the loss with respect to the tensor's values using the * backward() function
* Print the gradient values for the tensor pnt

## Building Autograd from scratch


## Forward propagation


Forward propagation, also known as forward pass or feedforward, is the process of moving input data through a neural network to generate an output. In other words, it's the process by which data is fed into the network, passes through its layers of neurons, and produces a prediction or classification.

In the context of building an automated differentiation library, the forward pass involves computing the value of the function being differentiated and simultaneously building a computational graph that represents the function and the dependencies between its inputs and outputs.

For each mathematical operation in the function (e.g. addition, multiplication, etc.), the forward pass creates a new node in the computational graph and connects it to the nodes corresponding to its inputs. The value of the output node is computed using the values of the input nodes and the mathematical operation.

As the computational graph is being built during the forward pass, each node in the graph stores information about its value and the mathematical operation that created it, as well as references to the nodes representing its inputs. This information will be used during the backward pass to compute gradients.

At the end of the forward pass, the output of the function being differentiated is returned along with the computational graph representing the function and its dependencies.

-----------------------

-----------------

## Backward propagation


Backpropagation, also known as backward propagation or backprop, is the process of computing the gradient of the loss function with respect to the weights of a neural network. It is used to update the weights of the network during training, in order to minimize the error between the predicted output and the actual output.

During backpropagation, the error between the predicted output and the actual output is propagated backwards through the network, layer by layer, in order to compute the gradient of the loss function with respect to the weights. This gradient is then used to update the weights of the network using an optimization algorithm such as stochastic gradient descent.

The purpose of backpropagation is to adjust the weights of the network in such a way that the error between the predicted output and the actual output is minimized. This process is repeated iteratively during training, until the network converges to a set of weights that produce accurate predictions.

![image](https://user-images.githubusercontent.com/83988379/235384228-21253678-3126-4996-9701-767c8cb96c02.png)


![image](https://user-images.githubusercontent.com/83988379/235384317-8183567f-178f-4080-8faa-44bb57f24878.png)


This is how to calculate $\frac{\partial \mathbb{L}}{\partial x_p}$:
<br><br>
$$
\mathbb{L} = \frac{1}{N} \sum_{i=0}^{N-1}[(x_{i} - x_{p})^{2} + (y_{i} - y_{p})^{2}]^{\frac{1}{2}}
\\
\mathbb{L} = C \sum_{i=0}^{N-1}{\mathbb{L}(x_i, y_i)}
\\
where\; C = \frac{1}{N}
$$

$$
\frac{\partial \mathbb{L}}{\partial x_p} = C \sum_{i=0}^{N-1}{\frac{\partial \mathbb{L}}{\partial x_p}(x_i, y_i)}
$$

$$
Required\; is\; to\; get: \;\;\frac{\partial \mathbb{L}}{\partial x_p},\;\; \frac{\partial \mathbb{L}}{\partial y_p}
$$

$$
\mathbb{L} (x_i, y_i) = [(x_{i} - x_{p})^{2} + (y_{i} - y_{p})^{2}]^{\frac{1}{2}}
$$

$$
Let \;\; I_x = x_i - x_p, \;\; I_y = y_i - y_p
\\
g_x = I_x^2, \;\; g_y = I_y^2
\\
M = g_x + g_y
\\
\mathbb{L} = M^\frac{1}{2}
$$

$$
\frac{\partial I_x}{\partial x_p} = -1,
\;\; \frac{\partial I_y}{\partial x_p} = 0
\\
\frac{\partial g_x}{\partial I_x} = 2 I_x,
\;\; \frac{\partial g_y}{\partial I_y} = 2 I_y
\\
\frac{\partial M}{\partial g_x} = 1,
\;\; \frac{\partial M}{\partial g_y} = 1
\\
\frac{\partial \mathbb{L}}{\partial M} = \frac{1}{2} M^\frac{-1}{2}
$$

$$
\frac{\partial \mathbb{L}}{\partial g_x} = \frac{\partial \mathbb{L}}{\partial M}\;\;.\;\frac{\partial M}{\partial g_x}
\\
\frac{\partial \mathbb{L}}{\partial g_x} = \frac{1}{2} M^\frac{-1}{2} \;. \; 1
\\
\frac{\partial \mathbb{L}}{\partial I_x} = \frac{\partial \mathbb{L}}{\partial g_x}\;\;.\;\frac{\partial g_x}{\partial I_x}
\\
\frac{\partial \mathbb{L}}{\partial g_x} = \frac{1}{2} M^\frac{-1}{2} \;. \; 2I_x
\\
\frac{\partial \mathbb{L}}{\partial g_x} = M^\frac{-1}{2} \; . \; I_x
\\
\frac{\partial \mathbb{L}}{\partial x} = \frac{\partial \mathbb{L}}{\partial I_x}\;\;.\;\frac{\partial I_x}{\partial x_p}
\\
\frac{\partial \mathbb{L}}{\partial x_p} = M^\frac{-1}{2} \; . \; I_x \; . \; -1
$$

$$
\frac{\partial \mathbb{L}}{\partial x_p} = -M^\frac{-1}{2} \; . \; I_x
\\
\frac{\partial \mathbb{L}}{\partial x_p} = -((x_i - x_p)^2 + (y_i - y_p)^2)^\frac{-1}{2} \;\; . \;\; (x_i - x_p) = \frac{\partial \mathbb{L}(x_i, y_i)}{\partial g_x}
$$

---------------------------

--------------------

To implement backward mode in Python from scratch, we can use a technique called "reverse-mode differentiation", also known as "backpropagation". Backpropagation works by breaking down a complex function into a series of simple, elementary operations, and then computing the derivative of the output with respect to each input in reverse order using the chain rule of calculus.



Here is a high-level overview of the steps involved in implementing backpropagation from scratch:

Define a computation graph for the function you want to differentiate, where each node in the graph represents an elementary operation and the edges represent the flow of inputs and outputs between nodes.
During the forward pass, compute the output of the function and store intermediate values (e.g. input, output, and derivative values) at each node in the graph.
During the backward pass, start at the output node and recursively compute the derivative of the output with respect to each input in the graph using the chain rule of calculus, while also propagating the derivative values backwards through the graph.
Once the derivative values for each input have been computed, they can be used to update the parameters of the model via gradient descent or other optimization algorithms.
This process can be somewhat complex and requires a deep understanding of calculus and graph theory. However, implementing backpropagation from scratch is a great exercise for learning the fundamental principles of deep learning and gaining a deeper understanding of how automatic differentiation works in practice.

Following the reccurence relation: Grad[i] = Grad[i-1]* Local Gradient

or more accuaretly  Grad[node]  =  Grad[parent] * Local Gradient of the node 

<font color="green">Local Gradient: 
$\frac{\partial(\text{parent})}{\partial(\text{child})}$ </font>  

                                      

---------------

-------------------

## Operator Overloading in Py

Operator overloading in Python allows developers to define how operators like +, -, *, /, ==, !=, <, >, and many others work with custom classes. By defining special methods with double underscores (e.g. __add__, __sub__, __mul__, __eq__, __ne__, __lt__, __gt__, etc.) in a class, developers can specify what should happen when the corresponding operator is used with instances of that class.

For example, if you define a Vector class, you can define __add__ to add two vectors together, and then use the + operator to add instances of the Vector class. This makes it possible to write more expressive and natural-looking code that works with custom classes in the same way it works with built-in types like integers, floats, and strings.

Operator overloading can be a powerful tool, but it should be used judiciously. Overusing operator overloading can lead to code that is hard to read and understand, so it's important to use it only when it makes the code more clear and expressive.

In [20]:
class comp_node:
      def __init__(self, val , children = [] ,op = "assign"):
          self.val = val
          self.children = children
          self.grad = 0 
          self.op = op
          self.backward_prop = lambda : None
          self.typ = ""
      def __to_comp_node__(self,obj):
           if not isinstance(obj,comp_node):
               return comp_node(val = obj)
           else: 
               return obj      
      def __sub__(self,other):
           other = self.__to_comp_node__(other)
           ret = comp_node( val = self.val - other.val ,
                           children = [self,other],
                           op ="sub")
           def _backward_prop():
                      self.grad  += ret.grad * 1
                      other.grad += ret.grad * (-1)
           ret.backward_prop = _backward_prop 
           return ret  

      def __rsub__(self,other):
           other = self.__to_comp_node__(other)
           return other - self 
              
           
      def __add__(self,other):
           other = self.__to_comp_node__(other)
           ret = comp_node( val = self.val + other.val ,
                           children = [self,other],
                           op ="add")
           def _backward_prop():
                      self.grad  += ret.grad * 1
                      other.grad += ret.grad * 1
           ret.backward_prop = _backward_prop  
           return ret  
      def __radd__(self,other):
           other = self.__to_comp_node__(other)
           return other + self         
      def __mul__(self,other):
           other = self.__to_comp_node__(other)
           ret = comp_node( val = self.val * other.val ,
                           children = [self,other],
                           op ="mult")
           def _backward_prop():
                self.grad += ret.grad * other.val  ## y=x*z >> y'wrt x = d(x)*z = 1*z ,, y'wrt z = d(z)*x = 1*x ||  x ->self , z ->other
                other.grad += ret.grad * self.val  ## y'wrt z = d(z)*x = 1*x   
           return ret

      def __rmul__(self,other):
           other = self.__to_comp_node__(other)
           return other * self            
      def __pow__(self , exponent):
          if not isinstance(exponent , (int , float)):
              raise ValueError("Unsuppoerted Datatypes")
          ret = comp_node(val = self.val**exponent ,
                          children = [self],
                          op = f"power  {exponent}")         
          def _backward_prop():
                      self.grad  += ret.grad * (exponent * self.val**(exponent - 1))   # L = M ^(1/2)  --->  out > L -- self - > child -> M  --  out -> parent -> L |||  child += parent * Local  
          ret.backward_prop = _backward_prop  
          return ret

      def __eq__(self,other):
             return self.val == other.val  
      def __repr__(self):
           return (f"op: {self.op} || Val: {self.val:.4f} || # children: {len(self.children)} || grad = {self.grad} ")         

      def __truediv__(self, other):
              other = self.__to_comp_node__(other)
              if other.val == 0:
                raise ValueError("can not divide by zero (undefined)")
              out = comp_node(val=self.val / other.val, children=[self, other], op="div")
              def _backward_prop():
                # d(u/v)/dx = (v*du/dx - u*dv/dx) / v^2 ---> For the first operand 'self'    
                  self.grad += out.grad / other.val   # dl/dself = 1*(1/other) 
                # d(u/v)/dv = -u/(v^2) ----> For the second operand 'other'
                  other.grad -= out.grad * self.val / (other.val**2)  # # dl/dself = 1*(-1)(other)**-2 = 1*(-1) / ((other)**2) 
              out.backward_prop = _backward_prop
              return out
      def __rtruediv__(self, other):
              other = self.__to_comp_node__(other)
              return other / self   
          
      def sin(self):
              out = comp_node(val=np.sin(self.val), children=[self], op="sin")
              def _backward_prop():
                  self.grad += out.grad * np.cos(self.val)
              out.backward_prop = _backward_prop
              return out
          
      def cos(self):
              out = comp_node(val=np.cos(self.val), children=[self], op="cos")
              def _backward_prop():
                  self.grad -= out.grad * np.sin(self.val)
              out.backward_prop = _backward_prop
              return out
assert comp_node(val = 5  ).val == 5 , " Assignment Failed :/" 
assert (comp_node( val = 5) - comp_node(val = 3)).val == 2
assert (comp_node( val = 5) - 3).val == 2
assert (3 - comp_node( val = 5)).val == -2
assert (comp_node(val = 5)**2).val == 25
assert (comp_node( val = 5 )**2) == comp_node( val = 25)
assert (comp_node(val = 5)+comp_node(val = 3)).val == 8
assert (comp_node(val = 5) + 3).val == 8  
assert (3 + comp_node(val = 5)).val == 8  
assert (comp_node(val = 5) * 3).val == 15  
assert (3 * comp_node(val = 5)).val == 15
assert ( comp_node(val = 15)/3).val == 5
assert ( 15 / comp_node(val = 3)).val == 5
####################################


data_x,data_y = generate_pnts(N=1)
x_p,y_p = comp_node(val=0.3),comp_node(val=0.3) 
def loss_graph(x_p,y_p,data_x,data_y):
          I_x,I_y = x_p-data_x, y_p-data_y
          g_x,g_y = I_x**2,I_y**2
          M = g_x+g_y
          l = M**(0.5)
          return l,[l,M,g_x,g_y,I_x,I_y,x_p,y_p]


ans,rev_top_order = loss_graph(x_p,y_p,data_x[0],data_y[0])
# print(ans,"\n")  -> op: power: (0.5) || Val: 0.5472 || children: 1 
rev_top_order[0].grad = 1
print("Traversing the Graph nodes in reverse top sort order\n")
for i, node in enumerate(rev_top_order):
      node.backward_prop()
      print(i , node)



Traversing the Graph nodes in reverse top sort order

0 op: power  0.5 || Val: 0.5472 || # children: 1 || grad = 1 
1 op: add || Val: 0.2994 || # children: 2 || grad = 0.9137222319490423 
2 op: power  2 || Val: 0.1043 || # children: 1 || grad = 0.9137222319490423 
3 op: power  2 || Val: 0.1952 || # children: 1 || grad = 0.9137222319490423 
4 op: sub || Val: -0.3229 || # children: 2 || grad = -0.5900849147094943 
5 op: sub || Val: -0.4418 || # children: 2 || grad = -0.8073411877467226 
6 op: assign || Val: 0.3000 || # children: 0 || grad = -0.5900849147094943 
7 op: assign || Val: 0.3000 || # children: 0 || grad = -0.8073411877467226 



The "__sub__" method is used when the subtraction operator is applied to the instance of the class, while "__rsub__" is used when the subtraction operator is applied to the other operand first, and the instance of the class is the second operand.

In the case of __rsub__, the backward propagation function is not necessary because the gradient of the first operand (the other operand) with respect to the output of the subtraction operation is already computed by its own __sub__ method. Therefore, we don't need to compute the gradients again in __rsub__.

In summary, we implemented the backward propagation function for __sub__ but not for __rsub__ because the gradients needed for __rsub__ are already computed by the __sub__ method of the other operand.

------------------

The code defines a class comp_node that represents a computational node in a computation graph.
 The class has the following attributes:
* val: the value of the node (Assigning)
* children: a list of child nodes ( similar to adjacecny list )
* grad: the gradient of the node (initialized to 0)
* op: the operation performed to compute the node's value (e.g., "add", "subtract", etc.)

The class has several special methods that overload the behavior of operators such as +, -, *, and **. These methods create new comp_node instances that represent the result of applying the corresponding operation to the operands.
The __to_comp_node__ method is a helper method that converts a given object to a comp_node instance if necessary.
The code includes several assertions to test the correctness of the implementation.

Overall, the code provides a basic implementation of automatic differentiation using the forward mode. It allows the user to define a computation graph and automatically compute the gradients of the nodes with respect to a given output node.